In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "10")
# spark.conf.set("spark.driver.memory", "8g")
# spark.conf.set("spark.driver.maxResultSize", "10g")
# spark.conf.set("spark.executor.memory", "10g")
spark

In [ ]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("./../sdg/data/retail-data/by-day/*.csv")

In [ ]:
staticDataFrame.printSchema()

In [ ]:
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [ ]:
from pyspark.sql.functions import window, desc, col

staticDataFrame\
    .selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
    .groupBy("CustomerId", window("InvoiceDate", "1 day"))\
    .sum("total_cost")\
    .show(5, truncate=False)

In [ ]:
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("../sdg/data/retail-data/by-day/*.csv")

In [ ]:
streamingDataFrame.isStreaming

In [ ]:
purchaseByCustomerPerHour = streamingDataFrame\
    .selectExpr(
        "CustomerId",
        "(UnitPrice * Quantity) as total_cost",
        "InvoiceDate"
    )\
    .groupBy(
        "CustomerId",
        window("InvoiceDate", "1 day")
    )\
    .sum("total_cost")

In [ ]:
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [ ]:
spark.sql("""
SELECT *
FROM customer_purchases
ORDER BY `sum(total_cost)` DESC
""").show(5, truncate=False)

In [ ]:
purchaseByCustomerPerHour.writeStream\
    .format("console")\
    .queryName("customr_purchases_2")\
    .outputMode("complete")\
    .start()

In [ ]:
from pyspark.sql.functions import date_format

preppedDataFrame = staticDataFrame\
    .na.fill(0)\
    .withColumn("day_of_week", date_format("InvoiceDate", "EEEE"))\
    .coalesce(5)

In [ ]:
trainDataFrame = preppedDataFrame\
    .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
    .where("InvoiceDate >= '2011-07-01'")

In [ ]:
display(trainDataFrame.count())
display(testDataFrame.count())

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer()\
    .setInputCol("day_of_week")\
    .setOutputCol("day_of_week_index")

In [ ]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder()\
    .setInputCol("day_of_week_index")\
    .setOutputCol("day_of_week_encoded")

In [ ]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
    .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
    .setOutputCol("features")

In [ ]:
from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
    .setStages([indexer, encoder, vectorAssembler])

In [ ]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [ ]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

In [ ]:
transformedTraining.cache()

In [ ]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans()\
    .setK(20)\
    .setSeed(1)

In [ ]:
kmModel = kmeans.fit(transformedTraining)

In [ ]:
kmModel.computeCost(transformedTraining)

In [ ]:
transformedTest = fittedPipeline.transform(testDataFrame)

In [ ]:
kmModel.computeCost(transformedTest)